# Connecting to an API/Pulling in the Data and Cleaning/Formatting
For this section, I will work on performing at least 5 data transformation and/or cleansing steps to API Data.

From this source, I will extract the technical and fundamental ticker data that helps me understand the relationship between them and the performance of the stock. For the simplicity of the analysis, I will focus on Apple, Nvidia, and Tesla.

## Load necessary libraries

In [4]:
import pandas as pd
import os
import numpy as np
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

# import requests
# from bs4 import BeautifulSoup
pd.set_option('display.max_colwidth', None)
import re

import yfinance as yf

## Load and review the API data before further processing
Since the yfinance library allows us to pull stock data directly from Yahoo Finance using its API, I will leverage the library for this section. 

Since last time we extracted the US economy data from the Federal Reserve Bank of St.Louis as well as web scraped the world events from Wikipedia from 01/01/2020 to 09/01/2024, I will align with this date range in this section.

In [6]:
# Define the tickers 
tickers = ['AAPL','NVDA','TSLA'] 

# Define the date range 
start = '2020-01-01'
end = '2024-09-01'

# Load the data
stockdt = yf.download(tickers, start = start, end = end, actions=True) # Set actions to True to extract the dividends and stock splits as well

# Check the data
stockdt.head()

[*********************100%***********************]  3 of 3 completed


Price                      Adj Close                           Close            
Ticker                          AAPL      NVDA       TSLA       AAPL     NVDA   
Date                                                                            
2020-01-02 00:00:00+00:00  72.796021  5.973123  28.684000  75.087502  5.99775  \
2020-01-03 00:00:00+00:00  72.088295  5.877516  29.534000  74.357498  5.90175   
2020-01-06 00:00:00+00:00  72.662712  5.902165  30.102667  74.949997  5.92650   
2020-01-07 00:00:00+00:00  72.320976  5.973619  31.270666  74.597504  5.99825   
2020-01-08 00:00:00+00:00  73.484360  5.984824  32.809334  75.797501  6.00950   

Price                                Dividends                 High  ...   
Ticker                          TSLA      AAPL NVDA TSLA       AAPL  ...   
Date                                                                 ...   
2020-01-02 00:00:00+00:00  28.684000       0.0  0.0  0.0  75.150002  ...  \
2020-01-03 00:00:00+00:00  29.534000       0.0  0.0  0.0  75.144997  ...   
2020-01-06 00:00:00+00:00  30.102667       0.0  0.0  0.0  74.989998  ...   
2020-01-07 00:00:00+00:00  31.270666       0.0  0.0  0.0  75.224998  ...   
2020-01-08 00:00:00+00:00  32.809334       0.0  0.0  0.0  76.110001  ...   

Price                            Low       Open                       
Ticker                          TSLA       AAPL     NVDA       TSLA   
Date                                                                  
2020-01-02 00:00:00+00:00  28.114000  74.059998  5.96875  28.299999  \
2020-01-03 00:00:00+00:00  29.128000  74.287498  5.87750  29.366667   
2020-01-06 00:00:00+00:00  29.333332  73.447502  5.80800  29.364668   
2020-01-07 00:00:00+00:00  30.224001  74.959999  5.95500  30.760000   
2020-01-08 00:00:00+00:00  31.215334  74.290001  5.99400  31.580000   

Price                     Stock Splits               Volume              
Ticker                            AAPL NVDA TSLA       AAPL       NVDA   
Date                                                                     
2020-01-02 00:00:00+00:00          0.0  0.0  0.0  135480400  237536000  \
2020-01-03 00:00:00+00:00          0.0  0.0  0.0  146322800  205384000   
2020-01-06 00:00:00+00:00          0.0  0.0  0.0  118387200  262636000   
2020-01-07 00:00:00+00:00          0.0  0.0  0.0  108872000  314856000   
2020-01-08 00:00:00+00:00          0.0  0.0  0.0  132079200  277108000   

Price                                 
Ticker                          TSLA  
Date                                  
2020-01-02 00:00:00+00:00  142981500  
2020-01-03 00:00:00+00:00  266677500  
2020-01-06 00:00:00+00:00  151995000  
2020-01-07 00:00:00+00:00  268231500  
2020-01-08 00:00:00+00:00  467164500  

[5 rows x 24 columns]

As we can see from the above, we have extracted the open, high, low, close, adjusted close, dividends, stock splits and volume of the three tickers.

Now let's extract the financial data for fundamentals. Since they have quarterley and yearly granularities, let's choose the quarterley ones for closer relationship with the granularity of the stock technical metrics, which is daily as shown above.

In [8]:
# Initialize dictionary to hold the fundamentals
fundamentals = {}

In [9]:
# Define the specific fields of interest for filtering
fundamental_fields = {
    'financials': ['Total Revenue','Gross Profit','Operating Expense','Research And Development', 'EBITDA'],
    'balance_sheet': ['Current Liabilities'],
    'cash_flow': ['Free Cash Flow'],
    'earnings': ['Basic EPS']
}

In [10]:
# Loop through the tickers for the financial data

for ticker in tickers:
    stock = yf.Ticker(ticker)

    # Retrieve quarterly financial data, meanwhile transposing the data to match the style of the daily stock prices data
    quarterly_financials = stock.quarterly_financials.T
    quarterly_balancesheet = stock.quarterly_balance_sheet.T
    quarterly_cashflow = stock.quarterly_cashflow.T
    quarterly_earnings = stock.quarterly_income_stmt.T

    # Filter the data by date range on the index
    quarterly_financials = quarterly_financials[(quarterly_financials.index >= start) & (quarterly_financials.index <= end)].sort_index()
    quarterly_balancesheet = quarterly_balancesheet[(quarterly_balancesheet.index >= start) & (quarterly_balancesheet.index <= end)].sort_index()
    quarterly_cashflow = quarterly_cashflow[(quarterly_cashflow.index >= start) & (quarterly_cashflow.index <= end)].sort_index()
    quarterly_earnings = quarterly_earnings[(quarterly_earnings.index >= start) & (quarterly_earnings.index <= end)].sort_index()

    # Create an empty DataFrame to hold selected fields
    financial_data = pd.DataFrame()  

    # Filter for only the fields of interest
    for category, fields in fundamental_fields.items():
        if category == "financials" and all(field in quarterly_financials.columns for field in fields):
            financial_data = pd.concat([financial_data, quarterly_financials[fields]], axis=1)
        elif category == "balance_sheet" and all(field in quarterly_balancesheet.columns for field in fields):
            financial_data = pd.concat([financial_data, quarterly_balancesheet[fields]], axis=1)
        elif category == "cash_flow" and all(field in quarterly_cashflow.columns for field in fields):
            financial_data = pd.concat([financial_data, quarterly_cashflow[fields]], axis=1)
        elif category == "earnings" and all(field in quarterly_earnings.columns for field in fields):
            financial_data = pd.concat([financial_data, quarterly_earnings[fields]], axis=1)

    # Store the filtered financial data for each ticker
    fundamentals[ticker] = financial_data

In [11]:
# Check the data extrated for AAPL
fundamentals['AAPL'].head()

,Total Revenue,Gross Profit,Operating Expense,Research And Development,EBITDA,Current Liabilities,Free Cash Flow,Basic EPS
2023-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-30,89498000000.0,40427000000.0,13458000000.0,7307000000.0,30653000000.0,145308000000.0,19435000000.0,1.47
2023-12-31,119575000000.0,54855000000.0,14482000000.0,7696000000.0,43221000000.0,133973000000.0,37503000000.0,2.19
2024-03-31,90753000000.0,42271000000.0,14371000000.0,7903000000.0,30736000000.0,123822000000.0,20694000000.0,1.53


In [12]:
# Check the data extract for NVDA
fundamentals['NVDA'].head()

,Total Revenue,Gross Profit,Operating Expense,Research And Development,EBITDA,Current Liabilities,Free Cash Flow,Basic EPS
2023-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.083
2023-07-31,13507000000.0,9462000000.0,2662000000.0,2040000000.0,7411000000.0,10334000000.0,6059000000.0,0.25
2023-10-31,18120000000.0,13400000000.0,2983000000.0,2294000000.0,10957000000.0,9101000000.0,7054000000.0,0.375
2024-01-31,22103000000.0,16791000000.0,3177000000.0,2465000000.0,14556000000.0,10631000000.0,11245000000.0,NaN
2024-04-30,26044000000.0,20406000000.0,3497000000.0,2720000000.0,17753000000.0,15223000000.0,14976000000.0,0.604


In [13]:
# Check the data extract for TSLA
fundamentals['TSLA'].head()

,Total Revenue,Gross Profit,Operating Expense,Research And Development,EBITDA,Current Liabilities,Free Cash Flow,Basic EPS
2023-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-30,23350000000.0,4178000000.0,2414000000.0,1161000000.0,3318000000.0,26640000000.0,849000000.0,0.58
2023-12-31,25167000000.0,4438000000.0,2374000000.0,1094000000.0,3484000000.0,28748000000.0,2063000000.0,2.49
2024-03-31,21301000000.0,3696000000.0,2525000000.0,1151000000.0,2875000000.0,29453000000.0,-2535000000.0,0.37


As we can see from the above, the data starts being available much later than the 2020-01-01 start date that we set up for the stock data, which limits the scope of the analysis later on, and we have to keep such limitation in mind.

## Data Transformations

### Step 1: Data Type Conversion
As can be observed from the stock data, the data index of the stockdt is in timestamp format with timezone information, which includes the time and timezone offset. Because the timestamp does not provide useful information as they are all zeros, let's convert to match the fundamental data date format.

In [17]:
# Check the stock data 
stockdt.index[:10]

DatetimeIndex(['2020-01-02 00:00:00+00:00', '2020-01-03 00:00:00+00:00',
               '2020-01-06 00:00:00+00:00', '2020-01-07 00:00:00+00:00',
               '2020-01-08 00:00:00+00:00', '2020-01-09 00:00:00+00:00',
               '2020-01-10 00:00:00+00:00', '2020-01-13 00:00:00+00:00',
               '2020-01-14 00:00:00+00:00', '2020-01-15 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Date', freq=None)

In [18]:
# Convert the index of the stock data to date format
stockdt.index = pd.to_datetime(stockdt.index.date)

In [19]:
# Check the data with the converted date index
stockdt.index[:10]

DatetimeIndex(['2020-01-02', '2020-01-03', '2020-01-06', '2020-01-07',
               '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-13',
               '2020-01-14', '2020-01-15'],
              dtype='datetime64[ns]', freq=None)

In [20]:
# Set back the index name
stockdt.index.name = 'Date'
stockdt.index[:10]

DatetimeIndex(['2020-01-02', '2020-01-03', '2020-01-06', '2020-01-07',
               '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-13',
               '2020-01-14', '2020-01-15'],
              dtype='datetime64[ns]', name='Date', freq=None)

### Step 2: Data Merge/Join
The stock data and the fundamental data both have the date as the index, which can be leveraged as the key to merge the two types of data. I will continue to work on the three tickers seperately for simplicity, and we can use loop later on for the same processing.

In [22]:
# Create a dictionary to store merged DataFrame for each ticker
merged_data_dict = {}

# Loop through each ticker to join stock data with the fundamental data on the date index
for ticker in tickers:
    # Find all comlumns that are related to the ticker in the stock data 
    stock_data_ticker = stockdt[[col for col in stockdt.columns if ticker in col]].copy()

    # Remove the ticker layer from the column index
    stock_data_ticker.columns = stock_data_ticker.columns.droplevel(1) # Level 0 is the Price and level 1 is the Ticker

    # Left join the stock data with the fundamental data on the date index
    merged_data = stock_data_ticker.join(fundamentals[ticker], how='left')

    # Store the joined data in the dictionary with the ticker as the key
    merged_data_dict[ticker] = merged_data

In [23]:
# Check the data 
for ticker, df in merged_data_dict.items():
    print(f'Merged data for {ticker}:')
    print(df.head(2))
    print('*' * 50)

Merged data for AAPL:
            Adj Close      Close  Dividends       High        Low       Open   
Date                                                                           
2020-01-02  72.796021  75.087502        0.0  75.150002  73.797501  74.059998  \
2020-01-03  72.088295  74.357498        0.0  75.144997  74.125000  74.287498   

            Stock Splits     Volume Total Revenue Gross Profit   
Date                                                             
2020-01-02           0.0  135480400           NaN          NaN  \
2020-01-03           0.0  146322800           NaN          NaN   

           Operating Expense Research And Development EBITDA   
Date                                                           
2020-01-02               NaN                      NaN    NaN  \
2020-01-03               NaN                      NaN    NaN   

           Current Liabilities Free Cash Flow Basic EPS  
Date                                                     
2020-01-02         

### Step 3: Check nulls and null handling
Since the fundamental data is very limited, I will not fill the null values with any statistical methods as not only because of the large gap between them and the stock data, but also because of the volatility of fundamentals for large cap companies. I will thus focus on the stock data.

In [25]:
# Check the nulls in each column of the DataFrame for each ticker

# Set up a dictionary to store null counts and percentage for each ticker
null_summary_dict = {}

# Loop through each ticker
for ticker, df in merged_data_dict.items():
    null_counts = df.isnull().sum() # Count nulls of each column

    null_percentage = (null_counts / len(df)) * 100 # Calculate null percentage to the dataframe

    null_sum = pd.DataFrame(
        {
            'Null Count': null_counts,
            'Null Percentage (%)': null_percentage
        }
    )

    null_summary_dict[ticker] = null_sum

In [26]:
# Check the results
null_summary_dict

{'AAPL':                           Null Count  Null Percentage (%)
 Adj Close                          0                  0.0
 Close                              0                  0.0
 Dividends                          0                  0.0
 High                               0                  0.0
 Low                                0                  0.0
 Open                               0                  0.0
 Stock Splits                       0                  0.0
 Volume                             0                  0.0
 Total Revenue                   1174                100.0
 Gross Profit                    1174                100.0
 Operating Expense               1174                100.0
 Research And Development        1174                100.0
 EBITDA                          1174                100.0
 Current Liabilities             1174                100.0
 Free Cash Flow                  1174                100.0
 Basic EPS                       1174           

From the statistics above, it looks weird that Apple and Tesla have 100% nulls for the fundamental data, which does not match what we saw in the earlier data extraction for fundamentals from the Yahoo Finance. Let's take a closer look.

In [28]:
# Let's look at Apple as an example
fundamentals['AAPL']

,Total Revenue,Gross Profit,Operating Expense,Research And Development,EBITDA,Current Liabilities,Free Cash Flow,Basic EPS
2023-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-30,89498000000.0,40427000000.0,13458000000.0,7307000000.0,30653000000.0,145308000000.0,19435000000.0,1.47
2023-12-31,119575000000.0,54855000000.0,14482000000.0,7696000000.0,43221000000.0,133973000000.0,37503000000.0,2.19
2024-03-31,90753000000.0,42271000000.0,14371000000.0,7903000000.0,30736000000.0,123822000000.0,20694000000.0,1.53
2024-06-30,85777000000.0,39678000000.0,14326000000.0,8006000000.0,28202000000.0,131624000000.0,26707000000.0,1.4


2023-09-30, 2023-12-31, 2024-03-31, and 2024-06-30 all landed on weekends and stock market does not open on weekends. Since companies release earnings during weekdays, and normally release the data some time after end of the quarter, I need to look up the earnings release date as the date and fill in with the financial data.

In [30]:
# Create dictionary of quarter-end to earning-date mapping for each ticker, based on the information from Yahoo Finance
earnings_release_dates = {
    'AAPL':{
        '2023-03-31':'2023-05-04',
        '2023-06-30':'2023-08-03',
        '2023-09-30':'2023-11-02',
        '2023-12-31':'2024-02-01',
        '2024-03-31':'2024-05-02',
        '2024-06-30':'2024-08-01'
    },
    'NVDA':{
        '2023-04-30':'2023-05-24',
        '2023-07-31':'2023-08-23',
        '2023-10-31':'2023-11-21',
        '2024-01-31':'2024-02-21',
        '2024-04-30':'2024-05-22',
        '2024-07-31':'2024-08-28'
    },
    'TSLA':{
        '2023-03-31':'2023-04-19',
        '2023-06-30':'2023-07-19',
        '2023-09-30':'2023-10-18',
        '2023-12-31':'2024-01-24',
        '2024-03-31':'2024-04-23',
        '2024-06-30':'2024-07-23'
    }
}

In [78]:
# Create a function to replace the quarter end dates with earnings release dates
def update_financial_dates(df, release_dates):
    # Convert the index to string for easy matching with the release dates dictionary keys
    df.index = df.index.strftime("%Y-%m-%d")
    
    # Map each quarter end date to its release date, if available
    df.index = df.index.map(lambda x: release_dates.get(x, x))  
    
    # Convert index back to datetime to keep consistent format
    df.index = pd.to_datetime(df.index)
    
    return df

In [32]:
# Apply the function to each ticker's fundamental data in the merged_data_dict
for ticker in merged_data_dict:
    # Get the release dates for the specific ticker
    ticker_release_dates = earnings_release_dates.get(ticker, {})
    
    # Update dates in the financial data using the specific ticker's earnings release dates
    fundamentals[ticker] = update_financial_dates(fundamentals[ticker], ticker_release_dates)
    
    # Select only the stock data columns (non-financial data)
    stock_data_ticker = merged_data_dict[ticker][[col for col in merged_data_dict[ticker].columns if col not in fundamentals[ticker].columns]]
    
    # Re-join stock data with updated financial data based on earnings release dates
    merged_data_dict[ticker] = stock_data_ticker.join(fundamentals[ticker], how="left")

In [84]:
# Rerun the null checking process

# Set up a dictionary to store null counts and percentage for each ticker
null_summary_dict = {}

# Loop through each ticker
for ticker, df in merged_data_dict.items():
    null_counts = df.isnull().sum() # Count nulls of each column

    null_percentage = (null_counts / len(df)) * 100 # Calculate null percentage to the dataframe

    null_sum = pd.DataFrame(
        {
            'Null Count': null_counts,
            'Null Percentage (%)': null_percentage
        }
    )

    null_summary_dict[ticker] = null_sum

# Check the null checking results
for ticker, summary in null_summary_dict.items():
    print(f'Null summary for {ticker}:')
    print(summary)
    print('*' * 50)

Null summary for AAPL:
                          Null Count  Null Percentage (%)
Adj Close                          0             0.000000
Close                              0             0.000000
Dividends                       1155            98.381601
High                               0             0.000000
Low                                0             0.000000
Open                               0             0.000000
Stock Splits                    1173            99.914821
Volume                             0             0.000000
Total Revenue                   1170            99.659284
Gross Profit                    1170            99.659284
Operating Expense               1170            99.659284
Research And Development        1170            99.659284
EBITDA                          1170            99.659284
Current Liabilities             1170            99.659284
Free Cash Flow                  1170            99.659284
Basic EPS                       1170            9

In [34]:
# Check specific examples on Apple as an example, checking the date index when the 'Total Revenue' is not null
apple_df = merged_data_dict['AAPL']
apple_df[apple_df['Total Revenue'].notnull()].index

DatetimeIndex(['2023-11-02', '2024-02-01', '2024-05-02', '2024-08-01'], dtype='datetime64[ns]', name='Date', freq=None)

According to the data validation, the date index mapping from the earnings date to the earnings release date is successful.

### Step 4: Data Type Conversion

In [37]:
# Check the data types
merged_data_dict['AAPL'].dtypes

Adj Close                   float64
Close                       float64
Dividends                   float64
High                        float64
Low                         float64
Open                        float64
Stock Splits                float64
Volume                        int64
Total Revenue                object
Gross Profit                 object
Operating Expense            object
Research And Development     object
EBITDA                       object
Current Liabilities          object
Free Cash Flow               object
Basic EPS                    object
dtype: object

All the fundamental fields are in object data type. Let's convert them to float data type for manipulation later.

In [39]:
for ticker, df in merged_data_dict.items():
    # Identify columns with object type
    object_columns = df.select_dtypes(include='object').columns

    # Convert to float64
    df[object_columns] = df[object_columns].apply(pd.to_numeric).astype('float64')

In [40]:
# Check the data types, again
merged_data_dict['AAPL'].dtypes

Adj Close                   float64
Close                       float64
Dividends                   float64
High                        float64
Low                         float64
Open                        float64
Stock Splits                float64
Volume                        int64
Total Revenue               float64
Gross Profit                float64
Operating Expense           float64
Research And Development    float64
EBITDA                      float64
Current Liabilities         float64
Free Cash Flow              float64
Basic EPS                   float64
dtype: object

### Step 5: Change the data presentation for nulls
As I have noticed, Dividends and Stock Splits have a lot of 0.0, which indicates there's no dividents or stock splits on that specific date. To avoid confusion or not to skew the ml models, let's convert the 0.0 to NaN. 

In [42]:
merged_data_dict['AAPL'].head()

,Adj Close,Close,Dividends,High,Low,Open,Stock Splits,Volume,Total Revenue,Gross Profit,Operating Expense,Research And Development,EBITDA,Current Liabilities,Free Cash Flow,Basic EPS
Date,,,,,,,,,,,,,,,,
2020-01-02,72.796021,75.087502,0.0,75.150002,73.797501,74.059998,0.0,135480400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,72.088295,74.357498,0.0,75.144997,74.125000,74.287498,0.0,146322800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06,72.662712,74.949997,0.0,74.989998,73.187500,73.447502,0.0,118387200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-07,72.320976,74.597504,0.0,75.224998,74.370003,74.959999,0.0,108872000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-08,73.484360,75.797501,0.0,76.110001,74.290001,74.290001,0.0,132079200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# Replace 0.0 with NaN for each ticker in the merged_data_dict
for ticker, df in merged_data_dict.items():
    df[['Dividends','Stock Splits']] = df[['Dividends','Stock Splits']].replace(0.0, np.nan)

In [44]:
merged_data_dict['AAPL'].head()

,Adj Close,Close,Dividends,High,Low,Open,Stock Splits,Volume,Total Revenue,Gross Profit,Operating Expense,Research And Development,EBITDA,Current Liabilities,Free Cash Flow,Basic EPS
Date,,,,,,,,,,,,,,,,
2020-01-02,72.796021,75.087502,NaN,75.150002,73.797501,74.059998,NaN,135480400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,72.088295,74.357498,NaN,75.144997,74.125000,74.287498,NaN,146322800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06,72.662712,74.949997,NaN,74.989998,73.187500,73.447502,NaN,118387200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-07,72.320976,74.597504,NaN,75.224998,74.370003,74.959999,NaN,108872000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-08,73.484360,75.797501,NaN,76.110001,74.290001,74.290001,NaN,132079200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Ethical Implications of the Data Wrangling

In the process of data wrangling for stock and fundamental financial data for Apple, Nvidia, and Tesla, I applied several transformations, such as merging datasets, replacing zero values in dividends and stock splits with NaN for ML later on, and aligning financial data to earnings release dates for analyzing stock reactions after the earnings later on. These changes ensure the data accurately reflects relevant financial information while minimizing skew in machine learning models. 

One key ethical consideration in this project is the potential impact of data manipulation on the interpretation of company performance and stock behavior, particularly given that alterations like substituting zero values with NaN could imply an assumption about data absence rather than inactivity. 

As these companies are publicly traded, my project must adhere to legal guidelines on financial data usage, especially regarding accurate representation and the removal of biases that could mislead. To mitigate ethical risks, I based all modifications on logic and assumptions that makes most sense to me, such as treating zero dividends as missing data rather than implying an event, and I maintained data credibility by sourcing it from Yahoo Finance, a reputable financial data provider. 

My approach prioritized ethical acquisition, transparency in transformations, and alignment with financial reporting standards to reduce potential misinterpretations or biases within the analysis.

## Putting the codes above into a function for module operation for the final milestone, and save it to a .py file

In [76]:
def stock_api_wrangling():
    tickers = ['AAPL','NVDA','TSLA'] 
    start = '2020-01-01'
    end = '2024-09-01'

    stockdt = yf.download(tickers, start = start, end = end, actions=True) 
    fundamentals = {}

    fundamental_fields = {
        'financials': ['Total Revenue','Gross Profit','Operating Expense','Research And Development', 'EBITDA'],
        'balance_sheet': ['Current Liabilities'],
        'cash_flow': ['Free Cash Flow'],
        'earnings': ['Basic EPS']
    }

    for ticker in tickers:
        stock = yf.Ticker(ticker)
    
        quarterly_financials = stock.quarterly_financials.T
        quarterly_balancesheet = stock.quarterly_balance_sheet.T
        quarterly_cashflow = stock.quarterly_cashflow.T
        quarterly_earnings = stock.quarterly_income_stmt.T
    
        quarterly_financials = quarterly_financials[(quarterly_financials.index >= start) & (quarterly_financials.index <= end)].sort_index()
        quarterly_balancesheet = quarterly_balancesheet[(quarterly_balancesheet.index >= start) & (quarterly_balancesheet.index <= end)].sort_index()
        quarterly_cashflow = quarterly_cashflow[(quarterly_cashflow.index >= start) & (quarterly_cashflow.index <= end)].sort_index()
        quarterly_earnings = quarterly_earnings[(quarterly_earnings.index >= start) & (quarterly_earnings.index <= end)].sort_index()
    
        financial_data = pd.DataFrame()  
    
        for category, fields in fundamental_fields.items():
            if category == "financials" and all(field in quarterly_financials.columns for field in fields):
                financial_data = pd.concat([financial_data, quarterly_financials[fields]], axis=1)
            elif category == "balance_sheet" and all(field in quarterly_balancesheet.columns for field in fields):
                financial_data = pd.concat([financial_data, quarterly_balancesheet[fields]], axis=1)
            elif category == "cash_flow" and all(field in quarterly_cashflow.columns for field in fields):
                financial_data = pd.concat([financial_data, quarterly_cashflow[fields]], axis=1)
            elif category == "earnings" and all(field in quarterly_earnings.columns for field in fields):
                financial_data = pd.concat([financial_data, quarterly_earnings[fields]], axis=1)
    
        fundamentals[ticker] = financial_data

    stockdt.index = pd.to_datetime(stockdt.index.date)
    stockdt.index.name = 'Date'

    merged_data_dict = {}

    for ticker in tickers:
        stock_data_ticker = stockdt[[col for col in stockdt.columns if ticker in col]].copy()
    
        stock_data_ticker.columns = stock_data_ticker.columns.droplevel(1) 
    
        merged_data = stock_data_ticker.join(fundamentals[ticker], how='left')
    
        merged_data_dict[ticker] = merged_data

    earnings_release_dates = {
        'AAPL':{
            '2023-03-31':'2023-05-04',
            '2023-06-30':'2023-08-03',
            '2023-09-30':'2023-11-02',
            '2023-12-31':'2024-02-01',
            '2024-03-31':'2024-05-02',
            '2024-06-30':'2024-08-01'
        },
        'NVDA':{
            '2023-04-30':'2023-05-24',
            '2023-07-31':'2023-08-23',
            '2023-10-31':'2023-11-21',
            '2024-01-31':'2024-02-21',
            '2024-04-30':'2024-05-22',
            '2024-07-31':'2024-08-28'
        },
        'TSLA':{
            '2023-03-31':'2023-04-19',
            '2023-06-30':'2023-07-19',
            '2023-09-30':'2023-10-18',
            '2023-12-31':'2024-01-24',
            '2024-03-31':'2024-04-23',
            '2024-06-30':'2024-07-23'
        }
    }

    for ticker in merged_data_dict:
        ticker_release_dates = earnings_release_dates.get(ticker, {})
        
        fundamentals[ticker] = update_financial_dates(fundamentals[ticker], ticker_release_dates)
        
        stock_data_ticker = merged_data_dict[ticker][[col for col in merged_data_dict[ticker].columns if col not in fundamentals[ticker].columns]]
        
        merged_data_dict[ticker] = stock_data_ticker.join(fundamentals[ticker], how="left")

    for ticker, df in merged_data_dict.items():
        object_columns = df.select_dtypes(include='object').columns
    
        df[object_columns] = df[object_columns].apply(pd.to_numeric).astype('float64')

    for ticker, df in merged_data_dict.items():
        df[['Dividends','Stock Splits']] = df[['Dividends','Stock Splits']].replace(0.0, np.nan)

    return merged_data_dict